In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [ ]:
label = 'm2h12'
time = 'apply_date'
COLS_TO_SHOW=150
ID = 'loanid'

In [ ]:
ex_list = []

In [ ]:
def h(content):
    display(HTML(content))

In [ ]:
def timehist1(col):
    N = 8000  # 选取8000个样本来做图
    tr[tr[label] == 0].set_index(time)[col].clip(0, N).plot(style='.', title='Hist ' + col, figsize=(15, 3))
    tr[tr[label] == 1].set_index(time)[col].clip(0, N).plot(style='.', title='Hist ' + col, figsize=(15, 3))
    te.set_index(time)[col].clip(0, N).plot(style='.', title=col + ' values over time (blue=no-fraud, orange=fraud, green=test)'
                                            , figsize=(15, 3))
    plt.show()
def _desc(data, col, label):
    d0 = data.describe().reset_index()
    d0.columns = [col, label]
    return d0.append({col:'unique values', label:data.unique().shape[0]}, ignore_index=True) \
             .append({col:'NaNs', label:data.isnull().sum()}, ignore_index=True) \
             .append({col:'NaNs share', label:np.round(data.isnull().sum() / data.shape[0], 4)}, ignore_index=True) \
    
def desc1(col):
    d0 = _desc(tr[col], col, 'Train')
    d1 = _desc(tr.loc[tr[label] == 1, col], col, 'Train fraud')
    d2 = _desc(tr.loc[tr[label] == 0, col], col, 'Train Not fraud')
    d3 = _desc(te[col], col, 'Test')
    #
    n_cut = int(te.shape[0] * 0.2)#这里是控制test后面的两部分分界线
    d4 = _desc(te.loc[:n_cut, col], col, 'Test public')#这里将test又分为了两部分，我可以再按月划分
    d5 = _desc(te.loc[n_cut:, col], col, 'Test private')
    dd = d0.merge(d1).merge(d2).merge(d3).merge(d4).merge(d5)
    display(dd)
    
    if col not in ex_list:
        h('<b>Most popular values (NaN = -999):</b>')
        N = 10
        d0 = tr[[label,col]].fillna(-999).groupby(col)[label].agg(['size','mean','sum']
                                                                 ).reset_index().sort_values('size', ascending=False).reset_index(drop=True)
        d1 = te[[ID,col]].fillna(-999).groupby(col)[ID].count().reset_index()
        dd = d0.merge(d1, how='left', on=col).head(N)
        dd = dd.rename({'size':'Count in train (desc)','mean':'Mean target','sum':'Sum target',ID:'Count in test'}, axis=1)
        display(dd)

        h('<b>Biggest fraud sum values in train (NaN = -999):</b>')
        dd = d0.sort_values('sum', ascending=False).reset_index(drop=True).head(N).merge(d1, how='left', on=col)
        dd = dd.rename({'size':'Count in train','mean':'Mean target','sum':'Sum target (desc)',ID:'Count in test'}, axis=1)
        display(dd)

def hist1(col):
    plt.figure(figsize=(15, 3))
    plt.subplot(121)
    plt.hist(tr[col], bins=70);
    plt.title('Train histogram: ' + col);
    plt.subplot(122)
    plt.hist(te[col], bins=70);
    plt.title('Test histogram: ' + col);
    plt.show()

def corr1(col):
    N = None #10000
    num_vars = [f for f in tr.columns if tr[f].dtype != 'object']
    trx = tr.head(N) if N is not None else tr.copy()
    corrs = trx[num_vars].corrwith(trx[col]).reset_index().sort_values(0, ascending=False).reset_index(drop=True).rename(
        {'index':'Column',0:'Correlation with ' + col}, axis=1)
    h('<b>Most correlated values with ' + col + ':</b>')
    trx = pd.concat([corrs.head(6), corrs.dropna().tail(5)])
    def linkx(val):
        return '<a href="#c_{}">{}</a>'.format(val, val) if val in included_cols else val
    trx['Column'] = trx['Column'].apply(linkx)
    h(trx.to_html(escape=False))
    
def numeric(col):
    timehist1(col)
    hist1(col)
    desc1(col)
    corr1(col) 
    
def categorical(col):
    desc1(col)

def proc(col):
    if col not in [label,time,ID]:
        categorical(col) if tr[col].dtype == 'object' else numeric(col)

In [ ]:
included_cols = list(tr.columns.values[:COLS_TO_SHOW])